----
This is now almost final, pending review and resolution of the remaining items here. See also [Code Review: TPC-H Q5 Example][] (I think there's nothing left anymore on this page).

<font color="red">**TODO**</font>:

- Link math terms to our math glossary on Read the Docs (when it is available). 
- Add links to our API documentation on Read the Docs for each code box.  

[Code Review: TPC-H Q5 Example]: <https://algebraix.atlassian.net/wiki/display/projX/Code+Review%3A+TPC-H+Q5+Example> (Algebraix Wiki: Code Review: TPC-H Q5 Example)

----

# TPC-H Query 5 &ndash; Introduction

This IPython notebook is the starting point of a series of tutorials that introduce how data algebra facilitates querying data from multiple sources and in different structures, on the example of a modified [TPC-H][] query. 

The tutorials assume basic familiarity with our library; we suggest working through our [Hello_World][] tutorial first. They also assume basic knowledge of relational data, RDF and XML. 

In some cases, later parts of the tutorial assume knowledge of concepts introduced in earlier parts, so it is best to work through them in the listed sequence:

- **[1-Introduction][] (this tutorial)**: Introduces this series of tutorials, TPC-H, the TPC-H query 5 and our modifications to it.
- [2-Tables][]: Introduces our representation of tabular data, on the example of CSV.
- [3-Graphs][]: Introduces our representation of RDF-style tabular graph data, on the example of Turtle.
- [4-Hierarchies][]: Introduces our representation of hierarchical data, on the example of XML.
- [5-Query][]: Brings it all together and explains the whole query.

[TPC-H]: <http://www.tpc.org/tpch/> (TPC-H Benchmark Main Page)
[Hello_World]: <../Hello_World.ipynb> (IPython Notebook: Hello World)
[1-Introduction]: <1-Introduction.ipynb> (IPython Notebook: TPC-H Query 5 - Introduction)
[2-Tables]: <2-Tables.ipynb> (IPython Notebook: TPC-H Query 5 - Tables)
[3-Graphs]: <3-Graphs.ipynb> (IPython Notebook: TPC-H Query 5 - Graphs)
[4-Hierarchies]: <4-Hierarchies.ipynb>  (IPython Notebook: TPC-H Query 5 - Hierarchies)
[5-Query]: <5-Query.ipynb>  (IPython Notebook: TPC-H Query 5 - Query)

## TPC-H

The TPC-H benchmark documentation can be found at [http://www.tpc.org/tpch/][]. It is a series of SQL queries that is provided together with standard data and is used to measure the performance of SQL databases. This is what TPC says about the TPC-H benchmark:

> The TPC Benchmark™H (TPC-H) is a decision support benchmark. It consists of a suite of business oriented ad-hoc queries and concurrent data modifications. The queries and the data populating the database have been chosen to have broad industry-wide relevance. This benchmark illustrates decision support systems that examine large volumes of data, execute queries with a high degree of complexity, and give answers to critical business questions.

The detailed documentation can be found in [tpch2.17.1.pdf][].

In this tutorial, we are not interested in showing or comparing performance; we chose a query from this benchmark because it is widely known in the data science community and provides a simple example close to real-world usage. Our library is meant to show how data algebra can be used to consistently represent data in any structure, and to process that data with operations that are mathematically defined.

[http://www.tpc.org/tpch/]: <http://www.tpc.org/tpch/> (TPC-H Benchmark Main Page)
[tpch2.17.1.pdf]: <http://www.tpc.org/tpc_documents_current_versions/pdf/tpch2.17.1.pdf> (TPC-H Benchmark Standard Specification)

## Query 5

Query 5 is described like this in the TPC-H documentation:

> **Local Supplier Volume Query (Q5)**
>
> This query lists the revenue volume done through local suppliers.
>
> **Business Question**
>
> The Local Supplier Volume Query lists for each nation in a region the revenue volume that resulted from lineitem 
transactions  in  which  the  customer  ordering  parts  and  the  supplier  filling  them  were  both  within  that  nation.  The query is run in order to determine whether to in stitute local distribution centers in a given region. The query considers only parts ordered in a given year. The query displays the  nations and revenue  volume in descending order by revenue. Revenue volume for all qualifying lineitems in a particular nation is defined as `sum(l_extendedprice * (1 - l_discount))`.

The query in SQL looks like this:

``` sql
    select
        n_name,
        sum(l_extendedprice * (1 - l_discount)) as revenue
    from
        customer, orders, lineitem,
        supplier,
        nation, region
    where
        c_custkey = o_custkey
        and l_orderkey = o_orderkey
        and l_suppkey = s_suppkey
        and c_nationkey = s_nationkey
        and s_nationkey = n_nationkey
        and n_regionkey = r_regionkey
        and r_name = 'MIDDLE EAST'
        and o_orderdate >= date '1994-01-01'
        and o_orderdate < date '1994-01-01' + interval '1' year
    group by
        n_name
    order by
        revenue desc;
```

(The single-letter/underscore prefixes indicate the table that contains the column.)

## Our Modifications

We modified the data for the purposes of this tutorial in the following way:

- The tables `customer`, `orders` and `lineitem` are provided as [CSV][] data. (The TPC-H data generator provides the tables in a different, proprietary format.) See [2-Tables - Representation of Tables][] for details.
- The data from the table `supplier` is provided as [RDF][] graph in [Turtle][]. See [3-Graphs - Representation of RDF Graphs as Clans][] for details.
- The data from the tables `nation` and `region` is provided as [XML][] document. See [4-Hierarchies - Representation of Hierarchical Data][] for details.

We then modified the query to accomodate the changes in data format, to produce the same result. Here is a short summary of the modifications to the original SQL query:

- We replaced the implicit joins in the `WHERE` clause with explicit joins and subqueries.
- Besides SQL subqueries (which return a table), we introduced two 'pseudo-subqueries' in non-SQL formats:
    - One is in SPARQL and queries the RDF graph `supplier`. The SPARQL `SELECT` query naturally returns a table that then can be (SQL-)joined with other tables.
    - The other is in XQuery and extracts data from the XML file `regions.xml`. The XQuery result is so formed that it results in a table-like XML structure, that we treat as if it were a table. (The XQuery statement is not executed as actual XQuery; it is there to illustrate what we are doing to the XML data, even though it is not in XML anymore at this point.)
- We also removed the final sorting of the results; this is not related to anything we want to show here.


[2-Tables - Representation of Tables]: <2-Tables.ipynb#Representation-of-Tables> (IPython Notebook: TPC-H Query 5 - Tables - Representation of Tables)
[3-Graphs - Representation of RDF Graphs as Clans]: <3-Graphs.ipynb#Representation-of-RDF-Graphs-as-Clans> (IPython Notebook: TPC-H Query 5 - Graphs - Representation of RDF Graphs as Clans)
[4-Hierarchies - Representation of Hierarchical Data]: <4-Hierarchies.ipynb#Representation-of-Hierarchical-Data>  (IPython Notebook: TPC-H Query 5 - Hierarchies - Representation of Hierarchical Data)
[CSV]: <https://tools.ietf.org/html/rfc4180> (RFC 4180 - CSV Format)
[RDF]: <http://www.w3.org/RDF/> (Resource Description Framework (RDF) - Overview)
[Turtle]: <http://www.w3.org/TR/turtle/> (RDF 1.1 Turtle - Terse RDF Triple Language)
[XML]: <http://www.w3.org/TR/REC-xml/> (Extensible Markup Language (XML) 1.0 (Fifth Edition))

## The Modified Query

This query, as-is, wouldn't run on any database, but it outlines what we are doing with this data:

```sql
SELECT
    nationname, 
    SUM(lineitem.extendedprice * (1 - lineitem.discount)) AS revenue
FROM (
    SELECT
        custkey, orderkey, suppkey, nationkey, nationname
    FROM (
        SELECT 
            custkey, suppkey, nationkey, nationname
        FROM (
            -- This is an SQL subquery. It extracts the customers in the selected 
            -- nations (which come from an XML pseudo-subquery) and returns the
            -- customer key 'custkey', nation key 'nationkey' and nation name 
            -- 'nationname'.
            SELECT 
                custkey, nationkey, nationname
            FROM 
                customer
            JOIN (
                -- This is a pseudo-subquery in XQuery. It extracts a list of nations
                -- for the desired region, and for every nation it returns an XML 'row' 
                -- with the columns 'nationkey' and 'nationname' (renamed from 'name').
                -- This forms an XML pseudo-'table' named 'nations' that is then joined 
                -- with 'customer'.
                for $x in doc("regions.xml")/regions/region[name="MIDDLE EAST"]/nation
                    return <nation>{$x/nationkey}<nationname>{data($x/name)}</nationname></nation>
            ) AS nations
            ON
                customer.nationkey = nations.nationkey
        ) AS customers_nations_projected
        JOIN (
            -- This is a pseudo-subquery in SPARQL. It extracts a table with the
            -- columns 'suppkey' and 'nationkey' from the RDF graph 'supplier'
            -- and injects them into the outer query as table 'supplier'.
            SELECT 
                ?suppkey, ?nationkey
            FROM 
                supplier
            WHERE {
                ?supplier <tpch:suppkey> ?suppkey .
                ?supplier <tpch:nationkey> ?nationkey .
            }
        ) AS supplier_solutions
        ON 
            supplier_solutions.nationkey = customers_joined_projected.nationkey
    ) AS result1
    JOIN (
        -- This is an SQL subquery. It filters the table 'orders' by the desired 
        -- date range and projects the columns 'orderkey' and 'custkey'.
        SELECT 
            orderkey, custkey
        FROM 
            orders
        WHERE 
            startdate <= orders.orderdate and orders.orderdate < enddate 
    ) AS orders_restricted_projected
    ON 
        result1.custkey = orders_restricted_projected.custkey
) AS result2
JOIN 
    lineitem
ON 
    lineitem.orderkey = result2.orderkey AND lineitem.suppkey = result2.suppkey
GROUP BY 
    nationname
```


## The Presentation

In the following sections of the tutorial, we first present our representations of the various data formats (relational/CSV, RDF/Turtle, hierarchical/XML) and our techniques to extract data from them, and finally put it all together in one data algebra representation of this query.

# Next Step

Continue with [2-Tables][]; it introduces our representation of tabular data, on the example of CSV.

[2-Tables]: <2-Tables.ipynb> (IPython Notebook: TPC-H Query 5 - Tables)

----
&copy; Copyright 2015 Algebraix Data Corporation

This file is part of [`algebraixlib`][] .

[`algebraixlib`][] is free software: you can redistribute it and/or modify it under the terms of [version 3 of the GNU Lesser General Public License][] as published by the [Free Software Foundation][].

[`algebraixlib`][] is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public License along with [`algebraixlib`][]. If not, see [GNU licenses][].

[`algebraixlib`]: <http://github.com/AlgebraixData/algebraixlib> (A Python library for data algebra)
[Version 3 of the GNU Lesser General Public License]: <http://www.gnu.org/licenses/lgpl-3.0-standalone.html> 
[Free Software Foundation]: <http://www.fsf.org/>
[GNU licenses]: <http://www.gnu.org/licenses/>